In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
# import rpy2.robjects as robjects
# from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import time
import pickle
import math
import matplotlib.pyplot as plt
from glob import glob

import importlib

import time
import requests

from joblib import Parallel, delayed
from joblib import parallel_backend
import seaborn as sns

import generateSHAP
import plotmeta

from scipy import interpolate
import statsmodels.api as sm
from os.path import exists
import matplotlib as mpl
import os
import json

import dask as dd

import dataframe_image

In [ ]:
loinctranslate = pd.read_csv('/home/hoyinchan/code/AKI_CDM_PY/loinc/LoincTable/Loinc.csv', low_memory=False)

In [ ]:
importlib.reload(plotmeta)
plotshapsn = plotmeta.plotmeta(order='single', interaction = 'nointeraction')
with open("/home/hoyinchan/blue/program_data/AKI_CDM_PY/MetaRegression/gamalltmp_single.pkl", "rb") as f:
    plotshap2 = pickle.load(f)
plotshapsn.copy(plotshap2)
plotshapsn.get_meta_data(filename="gamalltmp_single_noAUC.json")
plotshapsn.load_raw_data()

In [ ]:
plotshapsn.calculate_local_fit_all()

In [ ]:
plotshapsn.plot_meta1("LAB::LG6373-7(mmol/L)")

In [ ]:
filepath = "/home/hoyinchan/blue/program_data/AKI_CDM_PY/MetaRegression/"

In [ ]:
importlib.reload(plotmeta)
plotshapdi = plotmeta.plotmeta(order='double', interaction = 'interaction')
plotshapdi.get_meta_data()
# with open(filepath+"gamalltmp_double_interaction.pkl", "rb") as f:
#        plotshap2 = pickle.load(f)
# plotshapdi.copy(plotshap2)

In [ ]:
#plotshapdi.combine_interaction_json(path='/home/hoyinchan/blue/program_data/AKI_CDM_PY/MetaRegression/gam2d_tmp/')

In [ ]:
#plotshapdi.plot_fig3_cat(ext_obj=plotshapsn, numrow=8)

In [ ]:
#plotshapsn.plot_r2_rank()

In [ ]:
plotshapdi.gamdata_fitdata['double_interaction']['p.coeff']

In [ ]:
#plotshapdi.plot_fig3_cont(ext_obj=plotshapsn, numrow=24, plottype='addition')

In [ ]:
#plotshapdi.plot_fig3_cont(ext_obj=plotshapsn, numrow=24, plottype='interactonly')

In [ ]:
#plotshapdi.plot_fig3_cont(ext_obj=plotshapsn, numrow=24, plottype='full')

In [ ]:
#plotshapdi.plot_fig3_cont(ext_obj=plotshapsn, numrow=24, plottype='subtract')

In [ ]:
plot_fig3_cat(int_obj=plotshapdi, ext_obj=plotshapsn, numrow=24)

from matplotlib.lines import Line2D

In [ ]:
numrow=24

cin2 = plotshapdi.get_interaction_stat(ext_obj)
cin2 = cin2[~cin2['0_x'].str.contains('PX')]
cin2 = cin2[~cin2[-1].str.contains('PX')]
#cin2['r.sq_diff'] = cin2['r.sq']-cin2['r.sq_max_y']
cin2 = cin2[cin2['r.sq_diff']>0]
cin2 = cin2.sort_values('r.sq_max', ascending=False)
cin2 = cin2.groupby(-1).head(numrow).reset_index()
cin3a = cin2.sort_values(['r.sq_diff'], ascending=False).groupby([-1]).head(1).reset_index(drop=True).sort_values(['r.sq_diff'], ascending=False)
cin3b = cin2.sort_values(['r.sq_diff'], ascending=False).groupby(['0_x']).head(1).reset_index(drop=True).sort_values(['r.sq_diff'], ascending=False)

cin3b = cin3b.drop(-1,axis=1)
# Rename columns: _x to _tmp, _y to _x, and _tmp to _y
cin3b.columns = [col.replace('_x', '_tmp') for col in cin3b.columns]
cin3b.columns = [col.replace('_y', '_x') for col in cin3b.columns]
cin3b.columns = [col.replace('_tmp', '_y') for col in cin3b.columns]
# Rename the -1 column to '0_x'
cin3b[-1] = cin3b['0_y'].copy()

cin3 = pd.concat([cin3a,cin3b]).sort_values(['r.sq_diff'], ascending=False).drop_duplicates().head(numrow).reset_index(drop=True)
cin3 = cin3.sort_values('r.sq_diff',ascending=False).reset_index(drop=True)

# Create data table
cin3_table = cin3[[-1,'0_x','r.sq','r.sq_max_y','r.sq_diff']]
cin3_table.columns = ['Primary', 'Secondary', 'Multi r.sq', 'Uni r.sq', 'r.sq Diff']

cin3_table['Primary'] = cin3_table['Primary'].apply(plotshapdi.extra_translate_omop)
cin3_table['Secondary'] = cin3_table['Secondary'].apply(plotshapdi.extra_translate_omop)

In [ ]:
cin3_table

In [ ]:
df = cin3_table
outputname='Double'

# Create a new Document
doc = Document()
table = doc.add_table(rows=1, cols=len(cin3_table.columns))  # Adjusted for the DataFrame structure

# Insert the column names with center alignment and bold
hdr_cells = table.rows[0].cells
for i, column in enumerate(cin3_table.columns):
    hdr_cells[i].text = column
    hdr_cells[i].paragraphs[0].runs[0].font.bold = True
    hdr_cells[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER

# Insert the row data with center alignment for text and formatting for floats
for _, row in cin3_table.iterrows():
    row_cells = table.add_row().cells
    for i, value in enumerate(row):
        if isinstance(value, float):
            # Formatting float values with 6 decimal places
            row_cells[i].text = f"{value:.6f}"
        else:
            # Directly inserting text values
            row_cells[i].text = value
        row_cells[i].paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.save(outputname+'.docx') 